## Filtering Data

In [0]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SparkFiltering").getOrCreate()

In [3]:
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/dataviz-curriculum/day_1/wine.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("wine.csv"), sep=",", header=True)

# Show DataFrame
df.show()

+-------+--------------------+--------------------+------+-----+------------------+--------------------+-----------------+------------------+--------------------+
|country|         description|         designation|points|price|          province|            region_1|         region_2|           variety|              winery|
+-------+--------------------+--------------------+------+-----+------------------+--------------------+-----------------+------------------+--------------------+
|     US|This tremendous 1...|   Martha's Vineyard|    96|  235|        California|         Napa Valley|             Napa|Cabernet Sauvignon|               Heitz|
|  Spain|Ripe aromas of fi...|Carodorum Selecci...|    96|  110|    Northern Spain|                Toro|             null|     Tinta de Toro|Bodega Carmen Rod...|
|     US|Mac Watson honors...|Special Selected ...|    96|   90|        California|      Knights Valley|           Sonoma|   Sauvignon Blanc|            Macauley|
|     US|This spent 20

In [4]:
df.printSchema()

root
 |-- country: string (nullable = true)
 |-- description: string (nullable = true)
 |-- designation: string (nullable = true)
 |-- points: string (nullable = true)
 |-- price: string (nullable = true)
 |-- province: string (nullable = true)
 |-- region_1: string (nullable = true)
 |-- region_2: string (nullable = true)
 |-- variety: string (nullable = true)
 |-- winery: string (nullable = true)



In [6]:
# Order a DataFrame by ascending values
df.orderBy(df["points"].asc()).show(5)

+-------+--------------------+--------------------+------+-----+--------+------------------+--------------+-------+------------+
|country|         description|         designation|points|price|province|          region_1|      region_2|variety|      winery|
+-------+--------------------+--------------------+------+-----+--------+------------------+--------------+-------+------------+
|     US|The strong aroma ...|                null|  null| null|    null|              null|          null|   null|        null|
|  Italy|This offers gener...|                null|  null| null|    null|              null|          null|   null|        null|
|     US|This shows a deft...|                null|  null| null|    null|              null|          null|   null|        null|
|  Italy|"This nicely stru...| bitter almond an...|  null|   88|      24|Northeastern Italy|Venezia Giulia|   null|Pinot Grigio|
|     US|               Ripe |                null|  null| null|    null|              null|     

In [7]:
# Import functions
from pyspark.sql.functions import avg
df.select(avg("points")).show()

+-----------------+
|      avg(points)|
+-----------------+
|87.88834105383143|
+-----------------+



In [8]:
# Using SQL
df.filter("price<20").show()

+---------+--------------------+--------------------+------+-----+----------------+--------------------+--------------------+--------------------+--------------------+
|  country|         description|         designation|points|price|        province|            region_1|            region_2|             variety|              winery|
+---------+--------------------+--------------------+------+-----+----------------+--------------------+--------------------+--------------------+--------------------+
| Bulgaria|This Bulgarian Ma...|             Bergul̩|    90|   15|        Bulgaria|                null|                null|              Mavrud|        Villa Melnik|
|    Spain|Earthy plum and c...|              Amandi|    90|   17|         Galicia|       Ribeira Sacra|                null|             Menc�_a|      Don Bernardino|
|       US|There's a lot to ...|                null|    90|   18|      California|Russian River Valley|              Sonoma|          Chardonnay|            De

In [9]:
# Filter by price on certain columns
df.filter("price<20").select(['points','country', 'winery','price']).show()

+------+---------+--------------------+-----+
|points|  country|              winery|price|
+------+---------+--------------------+-----+
|    90| Bulgaria|        Villa Melnik|   15|
|    90|    Spain|      Don Bernardino|   17|
|    90|       US|            De Loach|   18|
|    91|       US|   Trinity Vineyards|   19|
|    91| Portugal|Adega Cooperativa...|   15|
|    86|       US|      Belle Ambiance|   10|
|    86| Portugal| Adega de Cantanhede|   12|
|    86|       US|            Parducci|   13|
|    86| Portugal|    Quinta do Portal|   10|
|    86|   France|               Rigal|   14|
|    86|       US|     The Naked Grape|   18|
|    86|   France|   Georges Vigouroux|   15|
|    86|   France|   Georges Vigouroux|   10|
|    86|       US| Martinez & Martinez|   17|
|    86|       US|           Ironstone|   12|
|    86|       US|       Leaping Horse|   10|
|    86|       US|        Kitchen Sink|   13|
|    86| Portugal|  Wines & Winemakers|   12|
|    86|Argentina|              Zo

### Using Python Comparison Operators

In [10]:
# Same results only this time using python
df.filter(df["price"] < 200).show()

+-------+--------------------+--------------------+------+-----+------------------+------------------+-----------------+------------------+--------------------+
|country|         description|         designation|points|price|          province|          region_1|         region_2|           variety|              winery|
+-------+--------------------+--------------------+------+-----+------------------+------------------+-----------------+------------------+--------------------+
|  Spain|Ripe aromas of fi...|Carodorum Selecci...|    96|  110|    Northern Spain|              Toro|             null|     Tinta de Toro|Bodega Carmen Rod...|
|     US|Mac Watson honors...|Special Selected ...|    96|   90|        California|    Knights Valley|           Sonoma|   Sauvignon Blanc|            Macauley|
|     US|This spent 20 mon...|             Reserve|    96|   65|            Oregon| Willamette Valley|Willamette Valley|        Pinot Noir|               Ponzi|
| France|This is the top w...|    

In [11]:
df.filter( (df["price"] < 200) | (df['points'] > 80) ).show()

+-------+--------------------+--------------------+------+-----+------------------+--------------------+-----------------+------------------+--------------------+
|country|         description|         designation|points|price|          province|            region_1|         region_2|           variety|              winery|
+-------+--------------------+--------------------+------+-----+------------------+--------------------+-----------------+------------------+--------------------+
|     US|This tremendous 1...|   Martha's Vineyard|    96|  235|        California|         Napa Valley|             Napa|Cabernet Sauvignon|               Heitz|
|  Spain|Ripe aromas of fi...|Carodorum Selecci...|    96|  110|    Northern Spain|                Toro|             null|     Tinta de Toro|Bodega Carmen Rod...|
|     US|Mac Watson honors...|Special Selected ...|    96|   90|        California|      Knights Valley|           Sonoma|   Sauvignon Blanc|            Macauley|
|     US|This spent 20

In [12]:
df.filter(df["country"] == "US").show()

+-------+--------------------+--------------------+------+-----+----------+--------------------+-----------------+------------------+--------------------+
|country|         description|         designation|points|price|  province|            region_1|         region_2|           variety|              winery|
+-------+--------------------+--------------------+------+-----+----------+--------------------+-----------------+------------------+--------------------+
|     US|This tremendous 1...|   Martha's Vineyard|    96|  235|California|         Napa Valley|             Napa|Cabernet Sauvignon|               Heitz|
|     US|Mac Watson honors...|Special Selected ...|    96|   90|California|      Knights Valley|           Sonoma|   Sauvignon Blanc|            Macauley|
|     US|This spent 20 mon...|             Reserve|    96|   65|    Oregon|   Willamette Valley|Willamette Valley|        Pinot Noir|               Ponzi|
|     US|This re-named vin...|              Silice|    95|   65|    Or